# MedGemma-4B Cardiology Finetuning (Unsloth + HF Push)

This notebook finetunes **google/medgemma-4b-it** on the same cardiology chunks dataset used in the Unsloth notebook, then **merges and pushes** the model to Hugging Face so teammates can load it directly without LoRA adapters.

**Why merge?** A merged 16-bit model is standalone and does not rely on Unsloth or adapter weights at inference time.

## Prerequisites
- You must accept MedGemma terms on its Hugging Face model page.
- You need a Hugging Face token with write access to your org/user.
- GPU recommended.

In [1]:
# Install Unsloth + training deps
!pip install -q "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install -q --no-deps trl peft accelerate bitsandbytes
!pip install -q transformers datasets huggingface_hub

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.5/376.5 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 31.4 MB/s et

In [2]:
import os
import getpass
import torch
from datasets import load_dataset
from huggingface_hub import login
from unsloth import FastLanguageModel
from trl import SFTTrainer, SFTConfig
from transformers import TrainingArguments

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
# Configuration
model_id = "google/medgemma-4b-it"
dataset_id = "ilyassacha/cardiologyChunks"
output_dir = "medgemma-4b-cardiology-lora"
merged_dir = "medgemma-4b-cardiology-merged-16bit"
hub_repo = "Ismailea04/medgemma-4b-cardiology-merged"

dataset_percentage = 0.05  # Use 10% of the dataset

max_seq_length = 1024
load_in_4bit = True
dtype = None  # auto (bf16 on Ampere+, fp16 on older GPUs)

per_device_train_batch_size = 1 # Reduced from 2
gradient_accumulation_steps = 2 # Increased from 4
max_steps = 60
learning_rate = 2e-6
seed = 3407

# ⚡ SPEED OPTIMIZATION OPTIONS ⚡
max_seq_length = 1024  # Reduced from 2048 for faster training
batch_size = 8  # Increase if GPU memory allows (was 4)
num_train_epochs = 0.2  # Quick training for validation (was 1)

# 🎯 REGULARIZATION (to prevent overfitting)
learning_rate = 5e-6  # REDUCED from 2e-4 (was way too high!)
weight_decay = 0.5  # L2 regularization - penalize large weights
warmup_ratio = 0.1  # Gradual warmup to stable learning

# Unsloth speed optimization
load_in_4bit = True
use_bnb_4bit = True
dtype = torch.float16

print(f"Using GPU: {torch.cuda.is_available()}")
print(f"Dataset fraction: {dataset_percentage * 100:.0f}%")

Using GPU: True
Dataset fraction: 5%


In [4]:
# ⚡ FIX MEMORY ISSUES ⚡

# 1. Reduce Batch Size drastically (from 8 to 2 or 1)
per_device_train_batch_size = 2

# 2. Increase Gradient Accumulation to compensate
# (If batch=2 and accum=4, your effective batch size is 8, same as before)
gradient_accumulation_steps = 4

# 3. Keep sequence length reasonable (1024 is fine, but lower to 512 if it still crashes)
max_seq_length = 1024

# 4. Ensure Gradient Checkpointing is ON (this saves massive memory)
# Unsloth usually handles this, but ensure your SFTConfig doesn't disable it.

In [5]:
from google.colab import userdata
token = userdata.get('HF_TOKEN')

In [6]:
# Hugging Face login
if token is None:
    token = getpass.getpass("Enter HF token (with write access): " )
login(token=token)

In [13]:
import os
os.environ['UNSLOTH_USE_MODELSCOPE'] = '1'

# Load base model in 4-bit
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_id,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    token=token,
)

==((====))==  Unsloth 2026.2.1: Fast Gemma3 patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


In [14]:
# Inject LoRA adapters (target all linear layers for best quality)
model = FastLanguageModel.get_peft_model(
    model,
    r=4,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
)

Unsloth: Making `base_model.model.model.vision_tower.vision_model` require gradients


In [15]:
# Load dataset (same as finetune_cardiology_unsloth)
dataset = load_dataset(dataset_id, split="train", streaming=False)
print(f"Dataset size (full): {len(dataset)}")

# Use a fraction of the dataset for faster training
sample_size = int(len(dataset) * dataset_percentage)
dataset = dataset.select(range(sample_size))
print(f"Dataset size (subset): {len(dataset)}")

Dataset size (full): 3197756
Dataset size (subset): 159887


In [16]:
def format_cardiology_example(example):
    text = example.get("chunk", "")
    if not text:
        text = example.get("text", "")
    if not text:
        text = example.get("content", "")
    formatted = f"""Below is a cardiology topic:

{text}
"""
    return {"text": formatted}

dataset = dataset.map(format_cardiology_example, remove_columns=dataset.column_names)
print(dataset[0]["text"][:300])

Below is a cardiology topic:

sinus rhythm. normal ecg.



In [17]:
training_args = SFTConfig(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size, # Uses the new value (2)
    gradient_accumulation_steps=gradient_accumulation_steps, # Uses the new value (4)
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    warmup_ratio=warmup_ratio,
    logging_steps=10, # Log more frequently to see if it's working
    max_steps = 500, # <--- FORCE STOP after 60 steps (takes ~10 mins)
    save_steps=200,
    optim="adamw_8bit",
    max_grad_norm=0.3,
    fp16=False, # Explicitly set False because Unsloth is forcing FP32 anyway
    bf16=False,
    seed=42,
    report_to="none",
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    args=training_args,
)

Unsloth: Switching to float32 training since model cannot work with float16


In [18]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 159,887 | Num Epochs = 1 | Total steps = 500
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 8,197,120 of 4,308,276,592 (0.19% trained)


Step,Training Loss
10,2.234600
20,2.265900
30,2.168600
40,2.222600
50,2.275900
60,2.321900
70,2.161300
80,2.246800
90,2.140700
100,2.165900


TrainOutput(global_step=500, training_loss=2.0629230461120605, metrics={'train_runtime': 4841.981, 'train_samples_per_second': 0.826, 'train_steps_per_second': 0.103, 'total_flos': 3.652524385446528e+16, 'train_loss': 2.0629230461120605, 'epoch': 0.025017512258581006})

## Save merged 16-bit model (recommended)
This merges LoRA into the base weights and creates a standalone HF model.
Do not use `trainer.save_model()` for this step.

In [19]:
# Push merged model to Hugging Face Hub
model.push_to_hub_merged(hub_repo, tokenizer, save_method="merged_16bit", token=token)

config.json: 0.00B [00:00, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...gy-merged/tokenizer.model: 100%|##########| 4.69MB / 4.69MB            

  ...ogy-merged/tokenizer.json: 100%|##########| 33.4MB / 33.4MB            

...

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...0002-of-00002.safetensors:   0%|          |  604kB / 3.64GB            

Unsloth: Merging weights into 16bit: 100%|██████████| 2/2 [07:03<00:00, 211.95s/it]


Unsloth: Merge process complete. Saved to `/content/Ismailea04/medgemma-4b-cardiology-merged`


In [ ]:
# Save merged model locally
model.save_pretrained_merged(merged_dir, tokenizer, save_method="merged_16bit")

## Test the merged model (standalone)
Run only this section if you just want to test the final model.
Set `model_source` to your HF repo or the local merged directory.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Choose one: HF repo or local folder
model_source = "Ismailea04/medgemma-4b-cardiology-merged"  # or "medgemma-4b-cardiology-merged-16bit"

tokenizer = AutoTokenizer.from_pretrained(model_source)
model = AutoModelForCausalLM.from_pretrained(
    model_source,
    # Changed from torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    # to dtype=torch.float32 to explicitly use float32 and fix the deprecation warning.
    dtype=torch.float32,
    device_map="auto",
)
model.eval()

def generate_response(prompt, max_new_tokens=256):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.inference_mode():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

test_prompts = [
    "What are the main causes of heart failure?",
    "Explain the pathophysiology of atrial fibrillation.",
    "How do you diagnose myocardial infarction?",
]

for prompt in test_prompts:
    print("=" * 60)
    print(f"Q: {prompt}")
    print("=" * 60)
    print(generate_response(prompt))
    print()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Q: What are the main causes of heart failure?


## Optional: Save GGUF for local inference
If your team uses llama.cpp / Ollama / LM Studio, you can export GGUF.

In [ ]:
# model.save_pretrained_gguf("medgemma-4b-cardiology-gguf", tokenizer, quantization_method="q8_0")